In [1]:

import os
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from splinter import Browser
import time
from selenium import webdriver

In [2]:
#data from https://github.com/BuzzFeedNews/nics-firearm-background-checks
background_df = pd.read_csv("./Resources/firearm.csv")
background_df


,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074,0.0,15.0,...,0.0,0.0,0.0,28.0,25.0,3.0,0.0,1.0,0.0,58549
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208,0.0,2.0,...,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,7400
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239,0.0,7.0,...,2.0,0.0,0.0,12.0,6.0,2.0,0.0,0.0,0.0,43789
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427,14.0,8.0,...,0.0,0.0,0.0,7.0,8.0,1.0,0.0,1.0,0.0,18508
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0,0.0,0.0,...,82.0,0.0,0.0,7708.0,3463.0,647.0,15.0,19.0,18.0,138358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,1998-11,Virginia,0.0,NaN,14.0,2.0,NaN,8,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
15726,1998-11,Washington,1.0,NaN,65.0,286.0,NaN,8,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361
15727,1998-11,West Virginia,3.0,NaN,149.0,251.0,NaN,5,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408
15728,1998-11,Wisconsin,0.0,NaN,25.0,214.0,NaN,2,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241


In [3]:
#reducing the DF
short_df = background_df[["month", "state","permit","permit_recheck","handgun","long_gun", "other", "multiple"]].copy()

short_df

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0
...,...,...,...,...,...,...,...,...
15725,1998-11,Virginia,0.0,NaN,14.0,2.0,NaN,8
15726,1998-11,Washington,1.0,NaN,65.0,286.0,NaN,8
15727,1998-11,West Virginia,3.0,NaN,149.0,251.0,NaN,5
15728,1998-11,Wisconsin,0.0,NaN,25.0,214.0,NaN,2


In [4]:
#drop row with NAN
reduced_df = short_df.dropna()
reduced_df

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0
...,...,...,...,...,...,...,...,...
4340,2016-02,Virginia,784.0,0.0,30085.0,15948.0,1133.0,0
4341,2016-02,Washington,15736.0,0.0,20583.0,11991.0,1832.0,863
4342,2016-02,West Virginia,3527.0,0.0,10746.0,7436.0,357.0,757
4343,2016-02,Wisconsin,9420.0,0.0,19465.0,12431.0,821.0,62


In [5]:
#renaming the columns

renamex_df = reduced_df.rename(columns={"month":"Date","state":"State","permit":"Permit","permit_recheck":"Permit_Rechecked","handgun":"Handgun","long_gun":"Long_Gun","other":"Other","multiple":"Multiple"})
Gun_Check=renamex_df[["Date","State","Permit","Permit_Rechecked","Handgun",
                                  "Long_Gun","Multiple","Other"]]

Gun_Check.to_csv("Resources/Checking.csv", index=False, header=True)

Gun_Check.tail()



,Date,State,Permit,Permit_Rechecked,Handgun,Long_Gun,Multiple,Other
4340,2016-02,Virginia,784.0,0.0,30085.0,15948.0,0,1133.0
4341,2016-02,Washington,15736.0,0.0,20583.0,11991.0,863,1832.0
4342,2016-02,West Virginia,3527.0,0.0,10746.0,7436.0,757,357.0
4343,2016-02,Wisconsin,9420.0,0.0,19465.0,12431.0,62,821.0
4344,2016-02,Wyoming,551.0,0.0,2287.0,2036.0,150,139.0


In [6]:

month_df=reduced_df['month'].str.split('-', expand=True)
year_df= month_df.rename(columns={0: "Year"})
year_df

,Year,1
0,2022,08
1,2022,08
2,2022,08
3,2022,08
4,2022,08
...,...,...
4340,2016,02
4341,2016,02
4342,2016,02
4343,2016,02


In [7]:
y_df= year_df[["Year"]].copy()
y_df

,Year
0,2022
1,2022
2,2022
3,2022
4,2022
...,...
4340,2016
4341,2016
4342,2016
4343,2016


In [8]:
joined_df=reduced_df.join(y_df)
joined_df

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,Year
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074,2022
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,373.0,208,2022
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239,2022
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427,2022
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,5743.0,0,2022
...,...,...,...,...,...,...,...,...,...
4340,2016-02,Virginia,784.0,0.0,30085.0,15948.0,1133.0,0,2016
4341,2016-02,Washington,15736.0,0.0,20583.0,11991.0,1832.0,863,2016
4342,2016-02,West Virginia,3527.0,0.0,10746.0,7436.0,357.0,757,2016
4343,2016-02,Wisconsin,9420.0,0.0,19465.0,12431.0,821.0,62,2016


In [9]:
cutting_df=joined_df.drop(["month"], axis=1)
cutting_df.head()


,state,permit,permit_recheck,handgun,long_gun,other,multiple,Year
0,Alabama,20271.0,326.0,18400.0,13679.0,1315.0,1074,2022
1,Alaska,291.0,16.0,3225.0,3005.0,373.0,208,2022
2,Arizona,8278.0,2029.0,19079.0,9304.0,1947.0,1239,2022
3,Arkansas,2585.0,334.0,6885.0,5697.0,516.0,427,2022
4,California,39274.0,14674.0,39236.0,25005.0,5743.0,0,2022


In [10]:
total_states = cutting_df["state"].nunique()
total_states

55

In [11]:
grouped_state_df = cutting_df.groupby(["state"])

In [12]:
total_permit =grouped_state_df["permit"].sum()

In [13]:
total_permit_recheck =grouped_state_df["permit_recheck"].sum()

In [14]:
total_handgun =grouped_state_df["handgun"].sum()

In [15]:
total_long_gun =grouped_state_df["long_gun"].sum()

In [16]:
total_other =grouped_state_df["other"].sum()

In [17]:
total_multiple =grouped_state_df["multiple"].sum()

In [18]:
Check_Summary_df = pd.DataFrame({"Total_permit" : total_permit,
      "Total_permit_recheck" : total_permit_recheck,
      "Total_handgun" : total_handgun,
      "Total_long_gun" : total_long_gun,
      "Total_other" : total_other,
      "Total_multiple" : total_multiple,
      })
#Check_Summary_df
FinalSummary_df=Check_Summary_df.drop(Check_Summary_df.index[[11,21,41,49]],axis=0)
Summary_df=FinalSummary_df.reset_index()
Summary_df.tail()

,state,Total_permit,Total_permit_recheck,Total_handgun,Total_long_gun,Total_other,Total_multiple
46,Virginia,70840.0,15351.0,2152444.0,1331494.0,255592.0,5
47,Washington,1280668.0,15880.0,1624937.0,1004646.0,257809.0,35132
48,West Virginia,210225.0,4727.0,542205.0,519123.0,27931.0,33954
49,Wisconsin,1240135.0,32132.0,1405853.0,1057908.0,85659.0,4133
50,Wyoming,41876.0,981.0,177227.0,184878.0,13369.0,10354


In [19]:
FinSummary_df =Summary_df.rename(columns={"state":"State"})
FinSummary_df.head()


,State,Total_permit,Total_permit_recheck,Total_handgun,Total_long_gun,Total_other,Total_multiple
0,Alabama,2101078.0,23815.0,1281150.0,924019.0,72211.0,60042
1,Alaska,17555.0,1243.0,243305.0,227508.0,25263.0,16442
2,Arizona,564451.0,49883.0,1385488.0,744420.0,118083.0,87491
3,Arkansas,293123.0,55598.0,580778.0,528715.0,30917.0,33579
4,California,3271982.0,236306.0,3439158.0,2433618.0,539235.0,11


# race firearms violence

In [20]:
#from https://www.kff.org/other/state-indicator/firearms-death-rate-by-raceethnicity/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D
#Deaths Due to Firearms per 100,000 Population by Race/Ethnicity
RaceC_df = pd.read_csv("./Resources/raw_data.csv")
RaceC_df.head()

,Location,Overall,White,Black,Hispanic,Asian or Pacific Islander,American Indian or Alaska Native
0,United States,13.6,11.6,31.8,7.9,2.9,18.7
1,Alabama,23.6,18.6,37,NSD,NaN,NaN
2,Alaska,23.5,20.6,NaN,NaN,NaN,40
3,Arizona,16.7,16.4,29.9,11.6,NSD,24.3
4,Arkansas,22.6,19.0,45.4,9.9,NaN,NaN


In [21]:
df = RaceC_df.fillna(0)
#df
df1 = df.replace(['NSD'],0)
#df1
drop6_df=df1.drop(labels=0)
drop6_df.set_index('Location',inplace=True)
#drop6_df.head()
Race_df= drop6_df.reset_index()
rename4_df = Race_df.rename(columns={"Location":"State"})
rename4_df.to_csv("Resources/Race_violence.csv", index=False, header=True)
rename4_df.head()

,State,Overall,White,Black,Hispanic,Asian or Pacific Islander,American Indian or Alaska Native
0,Alabama,23.6,18.6,37,0,0,0
1,Alaska,23.5,20.6,0,0,0,40
2,Arizona,16.7,16.4,29.9,11.6,0,24.3
3,Arkansas,22.6,19.0,45.4,9.9,0,0
4,California,8.5,8.0,27.1,7.1,2.7,10.4


# Deaths rate and deaths by state beetwen 2004-2020

In [22]:
#Deaths rate and deaths by state beetwen 2004-2020
# from https://www.cdc.gov/nchs/pressroom/sosmap/homicide_mortality/homicide.htm
death_df = pd.read_csv("./Resources/Deaths.csv")
death_df.head()

,Year,State,Death Rate,Deaths
0,2020,Alabama,14.2,654
1,2020,Alaska,7.3,55
2,2020,Arizona,7.5,525
3,2020,Arkansas,13.0,371
4,2020,California,6.1,2368


# Education ranking

In [23]:
# from https://worldpopulationreview.com/state-rankings/educational-attainment-by-state
education_df = pd.read_csv("./Resources/Education2022.csv")
education_df.head()

,State,High School or Higher,bachelors or Higher
0,Alabama,86.90%,26.20%
1,Alaska,93.10%,30.00%
2,Arizona,87.90%,30.30%
3,Arkansas,87.20%,23.80%
4,California,83.90%,34.70%


# Visiting the URL for the St Louis Fed site using splinter 

In [24]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.29M/6.29M [00:00<00:00, 8.20MB/s]


In [25]:
url ="https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2016-01-01"
browser.visit(url)
data0_df = pd.read_html(url)
df_0=data0_df[0]
df_0.columns = ["Unnamed: 0_level_1","Name","2016","Preceding Period","Year Ago from Period"]
df_0.head()

,Unnamed: 0_level_1,Name,2016,Preceding Period,Year Ago from Period
0,NaN,The United States,66657,64631,64631
1,NaN,Alabama,53314,50900,50900
2,NaN,Alaska,85494,85898,85898
3,NaN,Arizona,64468,59751,59751
4,NaN,Arkansas,51830,48944,48944


In [26]:
reduce0_df=df_0.drop(["Unnamed: 0_level_1","Year Ago from Period"], axis=1)
#reduce0_df.head()
rename0_df = reduce0_df.rename(columns={"Name":"State","2016":"Income_2016","Preceding Period":"Income_2015"})
rename0_df.head()


,State,Income_2016,Income_2015
0,The United States,66657,64631
1,Alabama,53314,50900
2,Alaska,85494,85898
3,Arizona,64468,59751
4,Arkansas,51830,48944


In [27]:
drop0_df=rename0_df.drop(labels=0)
drop0_df.set_index('State',inplace=True)
#drop0_df.head()
drop0_df.reset_index().head()

,State,Income_2016,Income_2015
0,Alabama,53314,50900
1,Alaska,85494,85898
2,Arizona,64468,59751
3,Arkansas,51830,48944
4,California,75235,72774


In [28]:

url_1 ="https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2018-01-01"
browser.visit(url_1)
data1_df = pd.read_html(url_1)
df_1=data1_df[0]
df_1.columns = ["Unnamed: 0_level_1","Name","2018","Preceding Period","Year Ago from Period"]
df_1.head()

,Unnamed: 0_level_1,Name,2018,Preceding Period,Year Ago from Period
0,NaN,The United States,68168,67571,67571
1,NaN,Alabama,53879,56219,56219
2,NaN,Alaska,74161,86196,86196
3,NaN,Arizona,67201,65984,65984
4,NaN,Arkansas,53712,54988,54988


In [29]:
reduce1_df=df_1.drop(["Unnamed: 0_level_1","Year Ago from Period"], axis=1)
#reduce1_df.head()
rename1_df = reduce1_df.rename(columns={"Name":"State","2018":"Income_2018","Preceding Period":"Income_2017"})
rename1_df.head()

,State,Income_2018,Income_2017
0,The United States,68168,67571
1,Alabama,53879,56219
2,Alaska,74161,86196
3,Arizona,67201,65984
4,Arkansas,53712,54988


In [30]:
drop1_df=rename1_df.drop(labels=0)
drop1_df.set_index('State',inplace=True)
#drop1_df.head()
drop1_df.reset_index().head()

,State,Income_2018,Income_2017
0,Alabama,53879,56219
1,Alaska,74161,86196
2,Arizona,67201,65984
3,Arkansas,53712,54988
4,California,76055,77410


In [31]:
url_2 ="https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2020-01-01"
browser.visit(url_2)
data2_df = pd.read_html(url_2)
df_2=data2_df[0]
df_2.columns = ["Unnamed: 0_level_1","Name","2020","Preceding Period","Year Ago from Period"]
df_2.head()

,Unnamed: 0_level_1,Name,2020,Preceding Period,Year Ago from Period
0,NaN,The United States,71186,72808,72808
1,NaN,Alabama,57243,59558,59558
2,NaN,Alaska,78236,83079,83079
3,NaN,Arizona,70220,74897,74897
4,NaN,Arkansas,53148,57799,57799


In [32]:
reduce2_df=df_2.drop(["Unnamed: 0_level_1","Year Ago from Period"], axis=1)
#reduce2_df.head()
rename2_df = reduce2_df.rename(columns={"Name":"State","2020":"Income_2020","Preceding Period":"Income_2019"})
rename2_df.head()

,State,Income_2020,Income_2019
0,The United States,71186,72808
1,Alabama,57243,59558
2,Alaska,78236,83079
3,Arizona,70220,74897
4,Arkansas,53148,57799


In [33]:
drop2_df=rename2_df.drop(labels=0)
drop2_df.set_index('State',inplace=True)
#drop2_df.head()
drop2_df.reset_index().head()

,State,Income_2020,Income_2019
0,Alabama,57243,59558
1,Alaska,78236,83079
2,Arizona,70220,74897
3,Arkansas,53148,57799
4,California,81278,82772


In [34]:
url_3 ="https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od="
browser.visit(url_3)
data3_df = pd.read_html(url_3)
df_3=data3_df[0]
df_3.columns = ["Unnamed: 0_level_1","Name","2021","Preceding Period","Year Ago from Period"]
df_3.head()


,Unnamed: 0_level_1,Name,2021,Preceding Period,Year Ago from Period
0,NaN,The United States,70784,71186,71186
1,NaN,Alabama,56929,57243,57243
2,NaN,Alaska,81133,78236,78236
3,NaN,Arizona,70821,70220,70220
4,NaN,Arkansas,50784,53148,53148


In [35]:
reduce3_df=df_3.drop(["Unnamed: 0_level_1","Preceding Period","Year Ago from Period"], axis=1)
#reduce3_df.head()
rename3_df = reduce3_df.rename(columns={"Name":"State","2021":"Income_2021"})
rename3_df.head()

,State,Income_2021
0,The United States,70784
1,Alabama,56929
2,Alaska,81133
3,Arizona,70821
4,Arkansas,50784


In [36]:
drop3_df=rename3_df.drop(labels=0)
drop3_df.set_index('State',inplace=True)
#drop3_df.head()
drop3_df.reset_index().head()

,State,Income_2021
0,Alabama,56929
1,Alaska,81133
2,Arizona,70821
3,Arkansas,50784
4,California,81575


# Merging the income dataframes

In [37]:
#merging
combined1_df= pd.merge(drop0_df,drop1_df, on ="State")
combined1_df.head()


,Income_2016,Income_2015,Income_2018,Income_2017
State,,,,
Alabama,53314,50900,53879,56219
Alaska,85494,85898,74161,86196
Arizona,64468,59751,67201,65984
Arkansas,51830,48944,53712,54988
California,75235,72774,76055,77410


In [38]:
combined2_df= pd.merge(combined1_df,drop2_df, on ="State")
combined2_df.head()

,Income_2016,Income_2015,Income_2018,Income_2017,Income_2020,Income_2019
State,,,,,,
Alabama,53314,50900,53879,56219,57243,59558
Alaska,85494,85898,74161,86196,78236,83079
Arizona,64468,59751,67201,65984,70220,74897
Arkansas,51830,48944,53712,54988,53148,57799
California,75235,72774,76055,77410,81278,82772


In [39]:
combined3_df= pd.merge(combined2_df,drop3_df, on ="State")
combined3_df.head()

,Income_2016,Income_2015,Income_2018,Income_2017,Income_2020,Income_2019,Income_2021
State,,,,,,,
Alabama,53314,50900,53879,56219,57243,59558,56929
Alaska,85494,85898,74161,86196,78236,83079,81133
Arizona,64468,59751,67201,65984,70220,74897,70821
Arkansas,51830,48944,53712,54988,53148,57799,50784
California,75235,72774,76055,77410,81278,82772,81575


In [40]:
F_df=combined3_df.reset_index()
F_df.head()

,State,Income_2016,Income_2015,Income_2018,Income_2017,Income_2020,Income_2019,Income_2021
0,Alabama,53314,50900,53879,56219,57243,59558,56929
1,Alaska,85494,85898,74161,86196,78236,83079,81133
2,Arizona,64468,59751,67201,65984,70220,74897,70821
3,Arkansas,51830,48944,53712,54988,53148,57799,50784
4,California,75235,72774,76055,77410,81278,82772,81575


In [41]:
#ordering the columns
final_combined_df = F_df[["State","Income_2015","Income_2016","Income_2017","Income_2018",
                                  "Income_2019","Income_2020","Income_2021"]]
final_combined_df.head()

,State,Income_2015,Income_2016,Income_2017,Income_2018,Income_2019,Income_2020,Income_2021
0,Alabama,50900,53314,56219,53879,59558,57243,56929
1,Alaska,85898,85494,86196,74161,83079,78236,81133
2,Arizona,59751,64468,65984,67201,74897,70220,70821
3,Arkansas,48944,51830,54988,53712,57799,53148,50784
4,California,72774,75235,77410,76055,82772,81278,81575


In [42]:
final_combined_df.to_csv("Resources/Income.csv", index=False, header=True)

# load the DFs on PostgreSQL

In [43]:
import psycopg2
from sqlalchemy import create_engine
from config import user,ps,host,port
import io

In [44]:
 # variables to populate the database connection string
db_user = user
db_password = ps
db_host = host
db_port = port

In [45]:
#confirming the database existence
db_name = "project3_db"

In [46]:
#Create engine to connect to project_db
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [47]:
conn=engine.connect()

In [48]:
death_df.to_sql('deaths', con=conn, if_exists='replace',index=False)

In [49]:
Gun_Check.to_sql('checking', con=conn, if_exists='replace',index=False)

In [50]:
education_df.to_sql('education2022', con=conn, if_exists='replace',index=False)

In [51]:
rename4_df.to_sql('raceviolence', con=conn, if_exists='replace',index=False)

In [52]:
final_combined_df.to_sql('income', con=conn, if_exists='replace',index=False)

# read from postgres

In [53]:
#Check for the existing tables
engine.table_names()

['deaths', 'checking', 'education2022', 'raceviolence', 'income']

In [55]:
#importing the dataframe from postgresql
value = pd.read_sql_table("income",engine)
value.head()

,State,Income_2015,Income_2016,Income_2017,Income_2018,Income_2019,Income_2020,Income_2021
0,Alabama,50900,53314,56219,53879,59558,57243,56929
1,Alaska,85898,85494,86196,74161,83079,78236,81133
2,Arizona,59751,64468,65984,67201,74897,70220,70821
3,Arkansas,48944,51830,54988,53712,57799,53148,50784
4,California,72774,75235,77410,76055,82772,81278,81575


In [56]:
value1= pd.read_sql_table("checking",engine)
value1.head()

,Date,State,Permit,Permit_Rechecked,Handgun,Long_Gun,Multiple,Other
0,2022-08,Alabama,20271.0,326.0,18400.0,13679.0,1074,1315.0
1,2022-08,Alaska,291.0,16.0,3225.0,3005.0,208,373.0
2,2022-08,Arizona,8278.0,2029.0,19079.0,9304.0,1239,1947.0
3,2022-08,Arkansas,2585.0,334.0,6885.0,5697.0,427,516.0
4,2022-08,California,39274.0,14674.0,39236.0,25005.0,0,5743.0


# converting df in json file for the plot

In [57]:
# convert the DF into a json  and save it
Gun_Check.to_json("Resources/GunCheck.json")

In [58]:
death_df.to_json("Resources/death.json")

In [59]:
education_df.to_json("Resources/education2022.json")

In [60]:
rename4_df.to_json("Resources/RaceViolence2020.json")

In [61]:
final_combined_df.to_json("Resources/Income2015_2021.json")